# Día 3 – IA conversacional – ¡también conocido como Chatbot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Cargar variables de entorno en un archivo llamado .env
# Imprimir los prefijos de clave para facilitar la depuración

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

OpenAI API Key existe y empieza por sk-proj-
Anthropic API Key existe y empieza por sk-ant-
Google API Key existe y empieza por AIzaSyBk


In [3]:
# Inicializar

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
system_message = "Eres un assistente muy útil"

# La estructura interna del historial de mensajes

Originalmente, gradio esperaba recibir una función llamada:

`chat(message, history)`

La cual debía recibir `history` en un formato particular, que debemos asignar al formato OpenAI antes de llamar a OpenAI:

```
[
{"role": "system", "content": "system message here"},
{"role": "user", "content": "first user prompt here"},
{"role": "assistant", "content": "the assistant's response"},
{"role": "user", "content": "the new user prompt"},
]
```

¡Pero Gradio se ha actualizado! Ahora pasará `history` en el formato exacto de OpenAI, perfecto para que lo enviemos directamente a OpenAI.

¡Así que nuestro trabajo se volvió más fácil!

Escribiremos una función `chat(message, history)` donde:
**message** es el mensaje que se debe usar
**history** es la conversación anterior, en formato OpenAI

Combinaremos el mensaje del sistema, el historial y el último mensaje, y luego llamaremos a OpenAI.

In [5]:
# ¡Ahora es solo una línea de código para preparar la entrada a OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("El historial es:")
    print(history)
    print("Y los mensajes son:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## ¡Y entonces entra la magia de Gradio!

In [6]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


El historial es:
[]
Y los mensajes son:
[{'role': 'system', 'content': 'Eres un assistente muy útil'}, {'role': 'user', 'content': 'Hola'}]
El historial es:
[{'role': 'user', 'metadata': {'title': None}, 'content': 'Hola'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': '¡Hola! ¿Cómo puedo ayudarte hoy?'}]
Y los mensajes son:
[{'role': 'system', 'content': 'Eres un assistente muy útil'}, {'role': 'user', 'metadata': {'title': None}, 'content': 'Hola'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': '¡Hola! ¿Cómo puedo ayudarte hoy?'}, {'role': 'user', 'content': 'Quiero comprar una corbata'}]
El historial es:
[{'role': 'user', 'metadata': {'title': None}, 'content': 'Hola'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': '¡Hola! ¿Cómo puedo ayudarte hoy?'}, {'role': 'user', 'metadata': {'title': None}, 'content': 'Quiero comprar una corbata'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': '¡Claro! Aquí tienes algunos consejos

In [10]:
system_message = "Eres un asistente útil en una tienda de ropa. \
Debes tratar de alentar gentilmente al cliente a que pruebe los artículos que están en oferta.\
Los sombreros tienen un 60 % de descuento y la mayoría de los demás artículos tienen un 50 % de descuento. \
Por ejemplo, si el cliente dice 'Quiero comprar un sombrero', \
podrías responder algo como 'Genial, tenemos muchos sombreros, incluidos varios que son parte de nuestro evento de rebajas'. \
Anima al cliente a comprar sombreros si no está seguro de qué comprar."

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [13]:
system_message += "\nSi el cliente pide zapatos, debes responder que los zapatos no están en oferta hoy, \
¡pero recuérdale al cliente que mire los sombreros!"

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


In [15]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    if 'cinturon' in message:
        messages.append({"role": "system", "content": "Para mayor contexto, la tienda no vende cinturones,\
        pero asegúrate de señalar otros artículos en oferta."})
    
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [16]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../business.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h2 style="color:#181;">Aplicaciones empresariales</h2>
<span style="color:#181;">Los asistentes conversacionales son, por supuesto, un caso de uso muy común para la IA de última generación, y los últimos modelos de vanguardia son notablemente buenos para conversaciones matizadas. Y Gradio facilita la creación de una interfaz de usuario. Otra habilidad crucial que cubrimos es cómo usar indicaciones para brindar contexto, información y ejemplos.
<br/><br/>
Piense en cómo podría aplicar un asistente de IA a su negocio y cree un prototipo. Utilice el mensaje del sistema para contextualizar su negocio y establecer el tono para el LLM.</span>
</td>
</tr>
</table>